In [2]:
import pandas as pd  


In [3]:
outgoing_df = pd.read_csv("../Data/Köln/importantStructureOutgoing.csv")

In [4]:
outgoing_df.groupby("base").count()

,link,type
base,,
50 Cent,29,29
Aaron Lebedeff,29,29
Act Up,28,28
Al Harewood,30,30
Alexander Jackson Davis,26,26
...,...,...
Werner Janssen (Komponist),29,29
Westminster Kennel Club Dog Show,28,28
William Beach Lawrence,29,29


In [5]:
views_df = pd.read_csv("../Data/Köln/views.csv")
len(views_df)

189188

Fill all empty dates in view_df with 0

In [6]:
all_dates = views_df[views_df["link"] == "50 Cent"]["date"]
# for link in views_df["link"]:
#   merged = 
#filled_views_df = list(map(lambda link: pd.merge(views_df[views_df["link"] == link], all_dates, how="right", on="date").fillna({"link": link, "views": 0}), list(views_df["link"])))

KeyboardInterrupt: 

In [7]:
all_dates = views_df[views_df["link"] == "50 Cent"]["date"]
filled_list = []
for link in views_df["link"].unique():
  filled_list.append(views_df[views_df["link"] == link].merge(all_dates,how="right", on="date").fillna({"link": link, "views": 0}))

filled_df = pd.concat(filled_list, ignore_index=True)

filled_df.to_csv("../Data/Köln/filled_views.csv", index=False)
  

In [8]:
filled_df

,link,date,views
0,FIFA-Weltfußballer des Jahres,2023070100,270.0
1,FIFA-Weltfußballer des Jahres,2023070200,871.0
2,FIFA-Weltfußballer des Jahres,2023070300,232.0
3,FIFA-Weltfußballer des Jahres,2023070400,215.0
4,FIFA-Weltfußballer des Jahres,2023070500,198.0
...,...,...,...
203572,COVID-19-Pandemie,2023092700,913.0
203573,COVID-19-Pandemie,2023092800,876.0
203574,COVID-19-Pandemie,2023092900,811.0
203575,COVID-19-Pandemie,2023093000,715.0


In [9]:
links = pd.read_csv("../Data/Köln/importantStructureOutgoing.csv")
links

,base,link,type
0,Charlie Barnett (Komiker),1954,outgoing
1,Charlie Barnett (Komiker),Komiker,outgoing
2,Charlie Barnett (Komiker),Afroamerikaner,outgoing
3,Charlie Barnett (Komiker),New York City,outgoing
4,Charlie Barnett (Komiker),West Virginia,outgoing
...,...,...,...
2545,Michael Blake (Musiker),Lounge Lizards,outgoing
2546,Michael Blake (Musiker),Billy Martin (Schlagzeuger),outgoing
2547,Michael Blake (Musiker),David Tronzo,outgoing
2548,Michael Blake (Musiker),Klarinette,outgoing


In [10]:
combination = filled_df.merge(links,on="link",how="right")
ml_output = []
for base in links["base"].sort_values().unique():
  ml_output += filled_df[filled_df["link"] == base].sort_values("date")["views"].to_list()
len(ml_output)

9300

In [11]:
ml_output

[684.0,
 577.0,
 537.0,
 465.0,
 527.0,
 809.0,
 557.0,
 498.0,
 486.0,
 493.0,
 485.0,
 553.0,
 556.0,
 805.0,
 906.0,
 762.0,
 552.0,
 560.0,
 603.0,
 514.0,
 573.0,
 541.0,
 585.0,
 566.0,
 520.0,
 511.0,
 462.0,
 526.0,
 537.0,
 609.0,
 528.0,
 524.0,
 576.0,
 561.0,
 571.0,
 685.0,
 619.0,
 552.0,
 624.0,
 606.0,
 500.0,
 762.0,
 887.0,
 750.0,
 536.0,
 526.0,
 483.0,
 509.0,
 533.0,
 639.0,
 560.0,
 514.0,
 538.0,
 624.0,
 590.0,
 681.0,
 575.0,
 590.0,
 513.0,
 478.0,
 527.0,
 495.0,
 626.0,
 547.0,
 574.0,
 533.0,
 498.0,
 1119.0,
 594.0,
 533.0,
 588.0,
 609.0,
 488.0,
 489.0,
 521.0,
 568.0,
 507.0,
 518.0,
 552.0,
 580.0,
 568.0,
 590.0,
 715.0,
 839.0,
 840.0,
 980.0,
 950.0,
 832.0,
 764.0,
 879.0,
 1272.0,
 1414.0,
 928.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 2.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 3.0,
 1.0,
 1.0,
 2.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 3.0,
 1.0,
 2.0,
 1.0,
 1.0,
 0.0,
 0.0,
 

In [12]:
combination = combination.sort_values(["base", "date"], ignore_index=True)
ml_input = []
ml_output = []
for base in combination["base"].unique():
  for date in combination["date"].unique():
    ml_input.append(combination[(combination["base"] == base) & (combination["date"] == date)])


In [14]:
ml_input[0]

,link,date,views,base,type
0,New York City,2023070100,1556.0,50 Cent,outgoing
1,Run-D.M.C.,2023070100,174.0,50 Cent,outgoing
2,Recording Industry Association of America,2023070100,22.0,50 Cent,outgoing
3,1975,2023070100,44.0,50 Cent,outgoing
4,Billboard Hot 100,2023070100,108.0,50 Cent,outgoing
5,Goldene Schallplatte,2023070100,221.0,50 Cent,outgoing
6,Vereinigte Staaten,2023070100,4378.0,50 Cent,outgoing
7,Deutsche Singlecharts,2023070100,228.0,50 Cent,outgoing
8,Präsidentschaftswahl in den Vereinigten Staate...,2023070100,186.0,50 Cent,outgoing
9,Candy Shop,2023070100,37.0,50 Cent,outgoing


In [18]:
ml_input_value = [e["views"].tolist() for e in ml_input]

In [19]:
len(ml_input_value)

9300